In [0]:
from pyspark.sql.functions import current_date

# Load the game table into a DataFrame
game_df = spark.table("dev_catalog.silver.game")

# Select distinct publishers from the game table
dim_publisher_df = game_df.select("Publisher").distinct()

# Add insert_date and update_date columns
dim_publisher_df = dim_publisher_df.withColumn("insert_date", current_date())
dim_publisher_df = dim_publisher_df.withColumn("update_date", current_date())

# Create or merge into the dim_publisher table
dim_publisher_df.createOrReplaceTempView("temp_dim_publisher")

merge_query = """
MERGE INTO dev_catalog.gold.dim_publisher AS target
USING temp_dim_publisher AS source
  ON target.publisher = source.publisher
WHEN MATCHED THEN
  UPDATE SET 
             target.update_date = current_date()
WHEN NOT MATCHED THEN
  INSERT ( publisher, insert_date, update_date)
  VALUES ( source.publisher, source.insert_date, source.update_date)
"""

spark.sql(merge_query)

# Display the dim_publisher table
#display(spark.table("dev_catalog.gold.dim_publisher"))